Name: Raghav Sharma<br>
Roll no: 25201313

In [2]:
import numpy as np
import pandas as pd
from numpy.linalg import eig, inv

In [3]:
data = pd.DataFrame({
'infection': ['viral','viral','viral','bacterial','bacterial','bacterial'],
'CRP': [20, 25, 30, 100, 110, 95],
'Temp': [37.2, 37.5, 38.0, 39.0, 39.5, 40.0]
})

In [4]:
data

,infection,CRP,Temp
0,viral,20,37.2
1,viral,25,37.5
2,viral,30,38.0
3,bacterial,100,39.0
4,bacterial,110,39.5
5,bacterial,95,40.0


In [5]:
means = data[['CRP','Temp']].mean()
stds = data[['CRP','Temp']].std(ddof=1)
data['zCRP'] = (data['CRP'] - means['CRP']) / stds['CRP']
data['zTemp'] = (data['Temp'] - means['Temp']) / stds['Temp']

In [6]:
virus = data[data['infection']=='viral'][['CRP','Temp']].to_numpy()
bact = data[data['infection']=='bacterial'][['CRP','Temp']].to_numpy()

In [7]:
cov_v = np.cov(virus.T)
cov_b = np.cov(bact.T)

In [8]:
n1, n2 = len(virus), len(bact)

In [9]:
W = (((n1-1)*cov_v) + ((n2-1)*cov_b)) / (n1+n2-2)

In [10]:
all_vals = data[['CRP','Temp']].to_numpy()
T = np.cov(all_vals.T)

In [11]:
B = T - W

In [12]:
S = inv(W) @ B

In [13]:
vals, vecs = eig(S)

In [14]:
idx = np.argsort(vals)[::-1]
eigvals = vals[idx]
eigvecs = vecs[:,idx]

In [15]:
LD1 = eigvecs[:,0] # First linear discriminant
LD2 = eigvecs[:,1]

In [16]:
print("Eigenvalues:", eigvals)
print("LD1 (coefficients for CRP, Temp):", LD1)
print("LD2:", LD2)

Eigenvalues: [44.40127075 -0.2       ]
LD1 (coefficients for CRP, Temp): [0.28016207 0.95995271]
LD2: [-0.02520938  0.99968219]


In [17]:
data['LD1_score'] = data[['CRP','Temp']].to_numpy() @ LD1

In [18]:
centroids = data.groupby('infection')['LD1_score'].mean()

In [19]:
cov_matrix = np.cov(all_vals.T)
pca_vals, pca_vecs = eig(cov_matrix)
pca_idx = np.argsort(pca_vals)[::-1]
PC1 = pca_vecs[:,pca_idx[0]]
data['PC1_score'] = all_vals @ PC1

In [20]:
final = data[['infection','CRP','Temp','zCRP','zTemp','LD1_score','PC1_score']]
print("\nFinal Table:")
print(final)


Final Table:
   infection  CRP  Temp      zCRP     zTemp  LD1_score   PC1_score
0      viral   20  37.2 -1.022323 -1.175454  41.313482   20.920488
1      viral   25  37.5 -0.904363 -0.910977  43.002279   25.926410
2      viral   30  38.0 -0.786403 -0.470182  44.883065   30.937314
3  bacterial  100  39.0  0.865043  0.411409  65.454363  100.940502
4  bacterial  110  39.5  1.100964  0.852204  68.735960  110.949854
5  bacterial   95  40.0  0.747083  1.293000  65.013505   95.966965


In [21]:
print("\nCentroid Scores (group means of LD1):")
print(centroids)


Centroid Scores (group means of LD1):
infection
bacterial    66.401276
viral        43.066275
Name: LD1_score, dtype: float64
